## Baseline Model Without Active Learning or Data Augmentation

To see what kind of performance our model achieves without any label querying or data augmentation, let's collect a random sample of data points, and see what accuracy we can achieve on the test set. For simplicity, we choose the size of the random sample to be 10,000 (~17% of the data). 

This baseline will help us have a rough sense of how much lift active learning and data augmentation add to our model's performance. 

In [1]:
import keras
from keras.models import Sequential
from keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Concatenate
)
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from data.get_data import load_mnist
import numpy as np

In [2]:
# collect the data
X_train, y_train = load_mnist('data/f_mnist_data', kind='train')
X_test, y_test = load_mnist('data/f_mnist_data', kind='t10k')

# minmax scaling
X_train, X_test = X_train/255.0, X_test/255.0

In [3]:
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# assemble random sample of data
sample_size = 10000
random_sample = np.random.choice(X_train.shape[0], sample_size, replace=False)
X_train_sample = X_train[random_sample]
y_train_sample = y_train[random_sample]

In [4]:
def create_keras_model():

    # model configs and hyperparams
    dim = (28,28,1)
    dropout_rate = .25
    
    model = Sequential([
        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=dim),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(dropout_rate),

        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),      
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(dropout_rate),
        
        Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),   
        MaxPooling2D(pool_size=(2, 2)),   
        Dropout(dropout_rate),
        
        Flatten(),
        
        Dense(1024, activation='relu'),
        Dropout(dropout_rate),
        
        Dense(512, activation='relu'),
        Dropout(dropout_rate),
        
        Dense(10, activation='softmax')
    ])

    adam = Adam(lr=0.001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    return model

classifier = KerasClassifier(create_keras_model)

In [5]:
# training params
epochs = 40

# fit model
results = classifier.fit(X_train_sample, y_train_sample, epochs=epochs)
test_acc = classifier.score(X_test, y_test)

2021-12-28 22:10:21.056574: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-28 22:10:21.211764: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/40
313/313 [==============================] - 22s 70ms/step - loss: 1.4301 - accuracy: 0.6055
Epoch 2/40
313/313 [==============================] - 22s 70ms/step - loss: 0.4866 - accuracy: 0.8261
Epoch 3/40
313/313 [==============================] - 23s 74ms/step - loss: 0.4317 - accuracy: 0.8382
Epoch 4/40
313/313 [==============================] - 22s 69ms/step - loss: 0.3807 - accuracy: 0.8623
Epoch 5/40
313/313 [==============================] - 22s 70ms/step - loss: 0.3456 - accuracy: 0.8709
Epoch 6/40
313/313 [==============================] - 22s 70ms/step - loss: 0.3153 - accuracy: 0.8796
Epoch 7/40
313/313 [==============================] - 22s 70ms/step - loss: 0.2887 - accuracy: 0.8936
Epoch 8/40
313/313 [==============================] - 24s 76ms/step - loss: 0.2737 - accuracy: 0.9028
Epoch 9/40
313/313 [==============================] - 26s 83ms/step - loss: 0.2701 - accuracy: 0.9025
Epoch 10/40
313/313 [==============================] - 27s 86ms/step - loss: 0.240

In [6]:
print(f"Number of total samples seen by baseline model during training: {X_train_sample.shape[0]}")

Number of total samples seen by baseline model during training: 10000


In [7]:
print(f"Accuracy on Test Set: {test_acc}")

Accuracy on Test Set: 0.8888000249862671


88.88% accuracy is a pretty solid basline to be working with. It's close to the target goal of 90% accuracy and we've only used ~17% of the data. But, if we can pick the examples we use to train our model more intelligently, it's possible that we can achieve even greater accuracy - and cross our 90% accuracy threshold - using fewer data points.  